In [ ]:
# Mount drive
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
#import libraries

import numpy as np
import tensorflow.keras as keras
import json
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

In [ ]:
# CONSTANTS

PATH_TO_DRIVE = "/content/drive/My Drive"
DATASET_PATH = PATH_TO_DRIVE + "/datasets/noise-speech-data-processed.json"
REDUCED_DATASET_PATH = PATH_TO_DRIVE + "/datasets/noise-speech-data-reduced-processed.json"

In [1]:
#Function to load processed audio data from JSON

def load_data(dataset_path):
  with open(dataset_path, "r") as fp:
    data = json.load(fp)

  # Convet list to numpy arrays
  inputs = np.array(data["mfcc"])
  targets = np.array(data["labels"])

  return inputs, targets

In [ ]:
def prepare_datasets(test_size, validation_size):

  # Load data
  X, y = load_data(DATASET_PATH)

  # create train-test split
  X_train, X_test, y_train, y_test = train_test_split(X,
                                                      y,
                                                      test_size=test_size)
  
  # create train-validation split
  X_train, X_validation, y_train, y_validation = train_test_split(X_train,
                                                                  y_train,
                                                                  test_size=validation_size)
  
  return X_train, X_validation, X_test, y_train, y_validation, y_test

In [ ]:
# Generates RNN-LSTM model

def build_model(input_shape):
  
  # Build network topology
  model = keras.Sequential()

  # 2 LSTM layers
  model.add(keras.layers.LSTM(8,
                             input_shape=input_shape,
                             return_sequences=True))
  model.add(keras.layers.LSTM(8))

  # dense layer
  model.add(keras.layers.Dense(4, activation="relu"))
  # model.add(keras.layers.Dropout(0.3))
  
  # Output layer
  model.add(keras.layers.Dense(2, activation="softmax"))

  return model # RNN-LSTM model

In [ ]:
# Build the train, test and validation sets
X_train, X_validation, X_test, y_train, y_validation, y_test = prepare_datasets(0.25,0.2)
print(X_train.shape)
print(X_validation.shape)
print(X_test.shape)

(1200, 216, 13)
(300, 216, 13)
(500, 216, 13)


In [ ]:
# extract input shape from input sets
input_shape = (X_train.shape[1], X_train.shape[2]) # 216,13

# Build the CNN model
model = build_model(input_shape)

In [ ]:
# Add optimizer
optimizer = keras.optimizers.Adam(learning_rate=0.0001)

# Compile the network
model.compile(optimizer=optimizer,
              loss="sparse_categorical_crossentropy",
              metrics=["accuracy"])

model.summary()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_12 (LSTM)               (None, 216, 8)            704       
_________________________________________________________________
lstm_13 (LSTM)               (None, 8)                 544       
_________________________________________________________________
dense_13 (Dense)             (None, 4)                 36        
_________________________________________________________________
dense_14 (Dense)             (None, 2)                 10        
Total params: 1,294
Trainable params: 1,294
Non-trainable params: 0
_________________________________________________________________


In [ ]:
# Train the CNN
model.fit(X_train, y_train,
          validation_data=(X_validation, y_validation),
          batch_size=8,
          epochs=30)

Epoch 1/30
150/150 [==============================] - 2s 15ms/step - loss: 0.2020 - accuracy: 0.9392 - val_loss: 0.2273 - val_accuracy: 0.9333
Epoch 2/30
150/150 [==============================] - 2s 15ms/step - loss: 0.1911 - accuracy: 0.9442 - val_loss: 0.2252 - val_accuracy: 0.9300
Epoch 3/30
150/150 [==============================] - 2s 15ms/step - loss: 0.1889 - accuracy: 0.9442 - val_loss: 0.2226 - val_accuracy: 0.9333
Epoch 4/30
150/150 [==============================] - 2s 15ms/step - loss: 0.1846 - accuracy: 0.9467 - val_loss: 0.2206 - val_accuracy: 0.9367
Epoch 5/30
150/150 [==============================] - 2s 15ms/step - loss: 0.1796 - accuracy: 0.9475 - val_loss: 0.2167 - val_accuracy: 0.9367
Epoch 6/30
150/150 [==============================] - 2s 15ms/step - loss: 0.1792 - accuracy: 0.9467 - val_loss: 0.2014 - val_accuracy: 0.9433
Epoch 7/30
150/150 [==============================] - 2s 15ms/step - loss: 0.1707 - accuracy: 0.9500 - val_loss: 0.2023 - val_accuracy: 0.9433

In [ ]:
# Evaluate the CNN on test set
test_error, test_accuracy = model.evaluate(X_test,
                                           y_test,
                                           verbose=1,
                                           batch_size=1)

print("Accuracy on test set is : {}".format(test_accuracy))

500/500 [==============================] - 4s 7ms/step - loss: 0.2201 - accuracy: 0.9380
Accuracy on test set is : 0.9380000233650208
